In [0]:
%run ./03_genie_config

In [0]:
import requests
import json
import time

# Genie API 呼び出し用の共通ヘッダ
HEADERS = {
    "Authorization": f"Bearer {DATABRICKS_TOKEN}",
    "Content-Type": "application/json",
}


def start_conversation(question: str) -> dict:
    """
    新しい Genie 会話を開始し、会話と最初のメッセージ情報を返す
    """
    url = f"{GENIE_API_BASE}/start-conversation"
    payload = {"content": question}

    resp = requests.post(url, headers=HEADERS, data=json.dumps(payload))
    resp.raise_for_status()
    return resp.json()


def send_message(conversation_id: str, question: str) -> dict:
    """
    既存の会話にメッセージを送信し、そのメッセージ情報を返す
    """
    url = f"{GENIE_API_BASE}/conversations/{conversation_id}/messages"
    payload = {"content": question}

    resp = requests.post(url, headers=HEADERS, data=json.dumps(payload))
    resp.raise_for_status()
    return resp.json()


def get_message(conversation_id: str, message_id: str) -> dict:
    """
    指定したメッセージの詳細（ステータスや添付情報など）を取得
    """
    url = f"{GENIE_API_BASE}/conversations/{conversation_id}/messages/{message_id}"

    resp = requests.get(url, headers=HEADERS)
    resp.raise_for_status()
    return resp.json()


def wait_until_completed(
    conversation_id: str,
    message_id: str,
    timeout_sec: int = 60,
    interval_sec: int = 5,
) -> dict:
    """
    メッセージのステータスが COMPLETED / FAILED / CANCELLED になるまでポーリング
    完了したらメッセージの最終状態（JSON）を返す
    """
    start = time.time()

    while True:
        msg = get_message(conversation_id, message_id)
        status = msg.get("status")

        if status in ("COMPLETED", "FAILED", "CANCELLED"):
            return msg

        if time.time() - start > timeout_sec:
            raise TimeoutError(f"Genie message did not complete within {timeout_sec} seconds")

        time.sleep(interval_sec)


def get_query_result(conversation_id: str, message_id: str, attachment_id: str) -> dict:
    """
    クエリ添付の結果を取得
    Genie のクエリ結果 API を叩き、生の JSON を返す
    """
    url = (
        f"{GENIE_API_BASE}/conversations/{conversation_id}"
        f"/messages/{message_id}/attachments/{attachment_id}/query-result"
    )

    resp = requests.get(url, headers=HEADERS)
    resp.raise_for_status()
    return resp.json()


def genie_query_result_to_df(result_json: dict):
    """
    Genie のクエリ結果(JSON)から Spark DataFrame を作成して返す

    想定するJSON構造:
    - statement_response.manifest.schema.columns ... カラム定義
    - statement_response.result.data_array       ... データ本体
    """
    stmt = result_json.get("statement_response") or {}

    # カラム情報を取り出す
    manifest = stmt.get("manifest") or {}
    schema = manifest.get("schema") or {}
    columns_meta = schema.get("columns") or []
    col_names = [c.get("name") for c in columns_meta]

    # 行データを取り出す
    result_part = stmt.get("result") or {}
    data_array = result_part.get("data_array") or []

    if not col_names or not data_array:
        raise ValueError("columns または data_array が空です。result_json を確認してください。")

    # Spark DataFrame を作成
    df = spark.createDataFrame(
        [tuple(row) for row in data_array],
        schema=col_names,
    )
    return df


# 読み込まれた関数一覧を表示
_defined_helpers = [
    "start_conversation(question: str) -> dict",
    "send_message(conversation_id: str, question: str) -> dict",
    "get_message(conversation_id: str, message_id: str) -> dict",
    "wait_until_completed(conversation_id: str, message_id: str, timeout_sec: int = 60, interval_sec: int = 5) -> dict",
    "get_query_result(conversation_id: str, message_id: str, attachment_id: str) -> dict",
    "genie_query_result_to_df(result_json: dict) -> DataFrame",
]

print("=== Genie API Functions Config ===")
print("以下の関数を利用できます：")
for sig in _defined_helpers:
    print(f"  - {sig}")